# 机器学习在天文学上的应用（分类)

- 从不可变主序列恒星中分离可变的RR Lyrae恒星

** 导入模块 **

In [41]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from astropy.io import fits 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import time

#warnings
import warnings
warnings.filterwarnings('ignore')

** 加载数据 **

In [42]:
hdulist = fits.open('../DATA/RRlyrae.fit')
rrlyrae = np.asarray(hdulist[1].data)
standards = np.load('../DATA/sdss_S82standards.npy')

**数据预处理 **

In [43]:
u_g = standards['mmu_u'] - standards['mmu_g']
g_r = standards['mmu_g'] - standards['mmu_r']
r_i = standards['mmu_r'] - standards['mmu_i']
i_z = standards['mmu_i'] - standards['mmu_z']

In [44]:
standards = standards[(u_g > 0.7) & (u_g < 1.35) &
                      (g_r > -0.15) & (g_r < 0.4) &
                      (r_i > -0.15) & (r_i < 0.22) &
                      (i_z > -0.21) & (i_z < 0.25)]

In [53]:
standards = standards[:1000]

**数据预处理 **

In [54]:
# get magnitudes and colors; split into train and test sets

mags_rr = np.vstack([rrlyrae[f + 'mag'] for f in 'ugriz'])
colors_rr = mags_rr[:-1] - mags_rr[1:]

mags_st = np.vstack([standards['mmu_' + f] for f in 'ugriz'])
colors_st = mags_st[:-1] - mags_st[1:]

In [55]:
# stack the two sets of colors together
X = np.vstack((colors_st.T, colors_rr.T))
y = np.zeros(X.shape[0])
y[-colors_rr.shape[1]:] = 1
y = y.astype(int)

In [56]:
X.shape[0]

1483

In [57]:
# np.random.RandomState函数
indices=[0,1,2,3]
np.random.RandomState(0).shuffle(indices)
indices

[2, 3, 1, 0]

** 训练集和测试集**

In [58]:
def split_samples(X, y, fractions=[0.75, 0.25], random_state=0):
    X = np.asarray(X)
    y = np.asarray(y)

    if X.shape[0] != y.shape[0]:
        raise ValueError("X and y should have the same leading dimension")
    n_samples = X.shape[0]

    fractions = np.asarray(fractions).ravel().cumsum()
    fractions /= fractions[-1]
    fractions *= n_samples
    N = np.concatenate([[0], fractions.astype(int)])
    N[-1] = n_samples  # in case of roundoff errors

    random_state = np.random.RandomState(random_state)
    indices = np.arange(len(y))
    random_state.shuffle(indices)

    X_divisions = tuple(X[indices[N[i]:N[i + 1]]]
                        for i in range(len(fractions)))
    y_divisions = tuple(y[indices[N[i]:N[i + 1]]]
                        for i in range(len(fractions)))

    return X_divisions, y_divisions

In [59]:
#N = len(y)
#Ntrain = int(N*0.75)
#X_train = X[:Ntrain]
#y_train = y[:Ntrain]

#X_test = X[Ntrain:]
#y_test = y[Ntrain:]
(X_train, X_test), (y_train, y_test) = split_samples(X, y, [0.75, 0.25], random_state=0)

** 计数 **

In [61]:
from  collections import Counter
import operator
#进行统计
a = dict(Counter(y))
#进行排序
b= sorted(a.items(), key=operator.itemgetter(1),reverse=True)
print(b)

[(0, 1000), (1, 483)]


In [62]:
N_tot = len(y)
N_st = np.sum(y == 0)
N_rr = N_tot - N_st
N_train = len(y_train)
N_test = len(y_test)
N_plot = 5000 + N_rr

** 训练模型 **

In [63]:
clf = RandomForestClassifier(n_estimators=100,random_state=33)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=33,
            verbose=0, warm_start=False)

** 模型预测 **

In [64]:
y_probs = clf.predict(X_test)

** 结果 **

In [65]:
sum = 0
for a, y in zip(y_probs, y_test):
    if a == y:
        sum = sum + 1
print ("%s of %s test values correct.\ntest accuracy: %f" % (sum, len(y_test), sum / len(y_test)))

361 of 371 test values correct.
test accuracy: 0.973046


** 不同模型预测 **

In [66]:
# Fit all the models to the training data
def compute_models(*args):
    names = []
    probs = []
    for classifier, kwargs in args:
        clf = classifier(**kwargs)
        clf.fit(X_train, y_train)
        y_probs = clf.predict(X_test)

        names.append(classifier.__name__)
        probs.append(y_probs)

    return names, probs

In [67]:
names, probs = compute_models((SVC, dict(C=1000.0, kernel='rbf')),
                              (LogisticRegression, dict(class_weight='auto')),
                              (KNeighborsClassifier, dict(n_neighbors=10)),
                              (DecisionTreeClassifier, dict(random_state=0, max_depth=12, criterion='entropy')),
                              (GaussianNB, {}),
                              (RandomForestClassifier, dict(n_estimators=1000,random_state=33)))


** 结果**

In [68]:
def result(names, probs):
    for i in range(len(names)):
        sum = 0
        for a, y in zip(probs[i], y_test):
            if a == y:
                sum = sum + 1
        print ("%s:\n %s of %s test values correct.\ntest accuracy: %f\n" % (names[i], sum, len(y_test), sum / len(y_test)))    

In [69]:
result(names, probs)

SVC:
 360 of 371 test values correct.
test accuracy: 0.970350

LogisticRegression:
 360 of 371 test values correct.
test accuracy: 0.970350

KNeighborsClassifier:
 360 of 371 test values correct.
test accuracy: 0.970350

DecisionTreeClassifier:
 360 of 371 test values correct.
test accuracy: 0.970350

GaussianNB:
 359 of 371 test values correct.
test accuracy: 0.967655

RandomForestClassifier:
 361 of 371 test values correct.
test accuracy: 0.973046

